# Solving Lorenz Dynamical System
Here, we solve the Dynamical System of Equations
$$ 
\begin{align*}
    \frac{\mathrm d x}{\mathrm d t} &= \sigma(y-x) \\
    \frac{\mathrm d y}{\mathrm d t} &= x(\rho - z) - y \\
    \frac{\mathrm d z}{\mathrm d t} &= xy - \beta z
\end{align*} 
$$
External packages used: 
- `numpy`
- `matplotlib`

In [ ]:
from typing import Callable
import numpy as np
from numpy._typing import NDArray
import matplotlib.pyplot as plt
import os
plt.rcParams.update({'font.size': 14})

In [ ]:
if "visuals" not in os.listdir():
    os.mkdir("visuals")

We first define a *state vector* $\mathbf X(t)$ such that
$$ 
\begin{align*}
    \mathbf X(t) &= \begin{bmatrix}
        x(t) \\
        y(t) \\
        z(t)
    \end{bmatrix}
\end{align*}
$$
Then, the Lorenz Equations can be modelled as a vector function $f(t, \mathbf X)$
$$ 
    \frac{\mathrm d\mathbf X}{\mathrm d t} = f(t, \mathbf X)
$$
Therefore, we can write this differential equation in the form of Range-Kutta 4th Order:
$$
    \mathbf X_{i+1} = \mathbf X_i + \frac{\Delta t}{6}(k_1 + 2k_2 + 2k_3 + k_4)
$$

In [ ]:
def rk4(X_i: NDArray, dt: float, f: Callable) -> NDArray:
    k1 = f(X_i)
    k2 = f(X_i + dt*k1/2)
    k3 = f(X_i + dt*k2/2)
    k4 = f(X_i + dt*k3)
    X_n = X_i + dt/6 * (k1 + 2*k2 + 2*k3 + k4)
    return X_n

The aforementioned Lorenz Equations:

In [ ]:
def lorenz(X: NDArray) -> NDArray:
    sigma = 10
    rho = 28
    beta = 8/3
    x, y, z = X

    dxdt = sigma*(y-x)
    dydt = x*(rho-z) - y
    dzdt = x*y - beta*z

    return np.array([dxdt, dydt, dzdt])

Initial conditions provided: $\mathbf X_0 = [1,\, 1,\, 1]$.

In [ ]:
X0 = np.array([1, 1, 1])
t_n = 50            # in seconds
dt = 0.001
N = int(t_n/dt)     # no. of elements/points

X = np.empty((N+1, 3))
X[0] = X0

for i in range(0, N):
    X[i+1] = rk4(X[i], dt, lorenz)

Plot of the trajectory visualized

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw={"projection":"3d"})

# ax.set_title("Phase space of Lorenz System")
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")

x, y, z = X.T
ax.plot(x, y, z)

fig.savefig("visuals/1-phase_space_lorenz.png")

Time series of the Lorenz system

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
time = np.arange(0, t_n+dt, dt)
ax.plot(time, x, label="x(t)")
ax.plot(time, y, label="y(t)")
ax.plot(time, z, label="z(t)")
ax.legend()
ax.set_ylabel("States")
ax.set_xlabel("Time (t)")
fig.savefig("visuals/2-time-series-lorenz")

### Lyapunov Exponent Analysis
Here, instead of solving in main program, solution is made into a function.

In [ ]:
def solve_lorenz(X0: NDArray[1], t0: float, t_n: float, dt: float) -> NDArray[2]:
    """
    Parameters
    ----------
    X0: NDArray
        initial position
    t0: float
        initial time
    t_n: float
        final time
    """
    N_i = int((t_n-t0)/dt)
    X_i = np.empty((N_i+1, 3))
    X_i[0] = X0
    for i in range(0, N_i):
        X_i[i+1] = rk4(X_i[i], dt, lorenz)
    
    return X_i

Function to compute the largest lyapunov exponent $\lambda$. See report for details

In [ ]:
def compute_largest_lyapunov(X_initial: NDArray):
    """
    X_initial: NDArray
        initial position
    """
    d0 = 1e-6    # Initial separation distance
    t0 = 0      # Initial time
    t_final = 1000
    t_step = 0.1   # Time step for reseting trajectories
    # dt = dt     # Using globally defined value for dt
    N_steps = int((t_final-t0)/t_step)

    X_step = np.array([d0, 0, 0])

    lyapunov_list = []
    X_new = X_initial
    for _ in range(N_steps):
        X_1 = solve_lorenz(X_new, 0, t_step, dt)
        X_2 = solve_lorenz(X_new + X_step, 0, t_step, dt)
        
        X_new = X_1[-1]
        X_pert = X_2[-1]
        distance_vector = X_pert - X_new
        distance = np.linalg.norm(distance_vector)
        lyapunov = np.log(distance / d0)
        lyapunov_list.append(lyapunov)

        # Update X_step in the direction of distance vector
        X_step = distance_vector/distance*d0

    return sum(lyapunov_list)/(len(lyapunov_list)*t_step)

In [ ]:
largest_lyapunov = compute_largest_lyapunov(X0)

Here, Largest Lyapunov is approximately 0.88936. For larger iterations, it gets closer to 0.90. The largest lyapunov being > 0 means it is a chaotic system.

In [ ]:
largest_lyapunov    # > 0 which implies chaotic behavior

### Animating different trajectories to show chaos

In [ ]:
from matplotlib import animation
from matplotlib.animation import PillowWriter

In [ ]:
X_1 = solve_lorenz(X0, 0, t_n, dt)
X_2 = solve_lorenz(X0 + np.array([0.1, 0, 0]), 0, t_n, dt)  # a small displacement
distances = np.linalg.norm(X_2 - X_1, axis=1)
t = np.arange(0, t_n+dt, dt)

fig = plt.figure(figsize=(16, 8))
dist_ax = fig.add_subplot(1, 2, 1)
path_ax = fig.add_subplot(1, 2, 2, projection='3d')

dist_ax.set_title("Distance between trajectories over Time")
dist_ax.set_xlim(0, t_n)
# dist_ax.set_ylim(0, np.max(distances))
dist_ax.set_xlabel("Time")
dist_ax.set_ylabel("Distance")

path_ax.set_title("Phase space for Multiple Trajectories")
path_ax.set_xlim(np.min(X_1.T[0])-1, np.max(X_1.T[0])+1)
path_ax.set_ylim(np.min(X_1.T[1])-1, np.max(X_1.T[1])+1)
path_ax.set_zlim(np.min(X_1.T[2])-1, np.max(X_1.T[2])+1)

path_ax.set_xlabel("X")
path_ax.set_ylabel("Y")
path_ax.set_zlabel("Z")

dist_plot, = dist_ax.plot([], [], label="Distance (in units)")
path1_plot, = path_ax.plot([], [], [], label="Trajectory 1")
path2_plot, = path_ax.plot([], [], [], label="Trajectory 2")

dist_ax.legend()
path_ax.legend()

frame_step = 100
total_frames = int(N/frame_step)

def animate(frame_num):
    index = frame_step*frame_num

    dist_plot.set_data(t[:index+1], distances[:index+1])
    dist_ax.relim()
    dist_ax.autoscale_view()

    x, y, z = X_1.T
    path1_plot.set_data(x[:index+1], y[:index+1])
    path1_plot.set_3d_properties(z[:index+1])

    x, y, z = X_2.T
    path2_plot.set_data(x[:index+1], y[:index+1])
    path2_plot.set_3d_properties(z[:index+1])

    return dist_plot, path1_plot, path2_plot

FPS = 50
ani = animation.FuncAnimation(fig, animate, frames=total_frames, interval=20, blit=True)
ani.save('lorenz_evolution.gif', writer="pillow", fps=FPS, dpi=100)
fig.savefig("2-lorenz_evolution.png")